# Relabel TMA cores

This notebook automatically labels the TMA cores from a JSON file with a standardized naming convention (R1C1, R1C2, etc). This ensures that on a TMA with many different FOVs across many different cores, no mistakes were made in naming. 

The inputs are manually generated JSON file with the selected cores, as well as a separate JSON defining the upper left hand corner and bottom right hand corner of the TMA grid. 

In [ ]:
import json
import os
from skimage.io import imread

from creed import tiling_utils

# suppress mpl deprecation
import warnings
from matplotlib.cbook import mplDeprecation
warnings.filterwarnings("ignore", category=mplDeprecation)

# 1. Generate the FOVs from a TMA spec file

You will first need to define the prefix to use for your input and output file names. This makes it easy to associate all of the files from a single run with one another. The default is `example_tma`.

The following input files are required for the script to run:

* `tma_corners_path`: the path to the JSON file containing FOVs defining the upper-left corner and bottom-right corner. It should be named `example_tma_corners.json`, where `example_tma` is replaced with the tma_prefix. 
* `manual_run_path`: the path to your run file with the manually selected FOVs. It should be named `example_tma_manual_run.json`.
* `slide_path`: the path to the slide image of the TMA

In [ ]:
# these variables need to be defined by the user
base_dir = "data_dir"
json_tiling_dir = os.path.join(base_dir, "json_tiling_data")

# define the prefix of each file
tma_prefix = 'example_tma'

# update with path to current slide image
slide_path = os.path.join(path_to_optical_image, 'tma_prefix')

In [ ]:
# these variables should be automatically defined; make sure they're named as described in the comment above 
tma_corners_path = os.path.join(json_tiling_dir, '%s_tma_corners.json' % tma_prefix)
auto_fov_names_path = os.path.join(json_tiling_dir, '%s_automatic_fov_names.json' % tma_prefix)
manual_run_path = os.path.join(json_tiling_dir, '%s_manual_run.json' % tma_prefix)
mapping_path = os.path.join(json_tiling_dir, '%s_mapping.json' % tma_prefix)
remapped_fov_path = os.path.join(json_tiling_dir, '%s_automatic_run.json' % tma_prefix)
moly_path = os.path.join(json_tiling_dir, '%s_moly_point.json' % tma_prefix)

In [ ]:
# Define TMA grid dimensions
tma_num_x = 4
tma_num_y = 7

# generate automatically named TMA
auto_fov_regions = tiling_utils.generate_tma_fov_list(
    tma_corners_path,
    tma_num_x,
    tma_num_y
)

# save the automatically-named TMA FOVs to centroids mapping
with open(auto_fov_names_path, 'w') as afrp:
    json.dump(auto_fov_regions, afrp)

# 2. Define the FOV locations and map to TMA-spec file FOV names

### Load the user-defined FOVs and slide image

In [ ]:
# load the user-defined set of FOVs in
with open(manual_run_path, 'r') as mfop:
    manual_fov_regions = json.load(mfop)

In [ ]:
# load the slide image in
slide_data = imread(slide_path)

### Map proposed tiles to their closest automatically-generated tile (Euclidean)

In [ ]:
manual_to_auto_map, manual_fovs_info, auto_fovs_info = tiling_utils.assign_closest_fovs(
    manual_fov_regions,
    auto_fov_regions
)

### Visualize and remap tiles

Usage notes:

* Proposed FOVs are drawn in red. Automatically-generated FOVs are drawn in blue.
* The selected proposed FOV and its mapped automatically-generated FOV are colored a darker shade.
* The `Manually-defined FOV` menu can be used to visualize current mappings to automatically-generated FOVs.
* The `Automatically-generated FOV` menu is used for re-mapping a FOV created by this script to one of yours.
* Additional validation parameters:
    * `check_dist`: set to a positive numeric value to notify proposed to automatically-generated FOV mappings at a distance greater than this value. If multiple pairs exist, they will be sorted by distance descending. Set to `None` to bypass this validation.
    * `check_duplicates`: set to `True` to notify of automatically-generated FOVs with more than 1 proposed FOV mapping to it.
    * `check_mismatches`: set to `True` to notify of proposed FOV names that don't matched their mapped automatically-generated FOV name
    * Notifications will be displayed in the `FOV pair validation checks` box
* When you're done generating your desired mapping, click the `Save mapping`. This will save your mapping to `mapping_path` specified earlier.

Note:

* The cells after this interactive widget (in the section `Use mapping to rename FOVs in manual_fov_regions`) are only to be run after you're satisfied with the mapping and clicked `Save mapping`.
* Prior to clicking `Save mapping` and re-running the cells in that section, ignore any error messages that may appear in that section.
* If you change your mind after saving the remapped FOVs, you can come back here to redo your mappings, save again, and re-run the code to remap the FOVs.

In [ ]:
# define which validation checks to display in the FOV pair validation checks box
check_dist = 50
check_duplicates = True
check_mismatches = True

In [8]:
%matplotlib widget
tiling_utils.tma_interactive_remap(
    manual_to_auto_map,
    manual_fovs_info,
    auto_fovs_info,
    slide_data,
    mapping_path,
    draw_radius=7,
    figsize=(7, 7),
    check_dist=check_dist,
    check_duplicates=check_duplicates,
    check_mismatches=check_mismatches
)

### Use proposed-to-auto mapping to rename FOVs in `manual_fov_regions`

In [ ]:
# load the mapping saved by the interactive visualization in
with open(mapping_path, 'r') as mp:
    mapping = json.load(mp)

In [ ]:
# whether to randomize the FOVs in remapped_fov_regions
randomize = True

# whether to insert Moly points between a specified interval of FOVs in remapped_fov_regions
moly_insert = True
moly_interval = 5

In [ ]:
# rename FOVs, randomize the order, and insert Moly points at a specified interval
remapped_fov_regions = tiling_utils.remap_and_reorder_fovs(
    manual_fov_regions,
    mapping,
    moly_path,
    randomize=randomize,
    moly_insert=moly_insert,
    moly_interval=moly_interval
)

In [ ]:
# save remapped_fov_regions
with open(remapped_fov_path, 'w') as rtp:
    json.dump(remapped_fov_regions, rtp)